In [1]:
from IPython import display
!pip install "grad-cam>=1.3.6" -q
!pip install -U openmim -q
!pip install -q /kaggle/input/frozen-packages-mmdetection/mmcv-2.0.1-cp310-cp310-linux_x86_64.whl

!git clone https://github.com/open-mmlab/mmpretrain.git
%cd mmpretrain
!mim install -e .

!mkdir checkpoint
!mkdir outputs
!mkdir data

display.clear_output()

In [2]:
import os
import shutil
import random

In [3]:
from mmpretrain import list_models, inference_model
list_models(task='Image Classification',pattern='swinv2')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


['swinv2-base-w16_3rdparty_in1k-256px',
 'swinv2-base-w16_in21k-pre_3rdparty_in1k-256px',
 'swinv2-base-w24_in21k-pre_3rdparty_in1k-384px',
 'swinv2-base-w8_3rdparty_in1k-256px',
 'swinv2-large-w16_in21k-pre_3rdparty_in1k-256px',
 'swinv2-large-w24_in21k-pre_3rdparty_in1k-384px',
 'swinv2-small-w16_3rdparty_in1k-256px',
 'swinv2-small-w8_3rdparty_in1k-256px',
 'swinv2-tiny-w16_3rdparty_in1k-256px',
 'swinv2-tiny-w8_3rdparty_in1k-256px']

In [4]:
!wget https://download.openmmlab.com/mmclassification/v0/swin-v2/swinv2-small-w8_3rdparty_in1k-256px_20220803-b01a4332.pth -P ./checkpoints
display.clear_output()

In [5]:
from mmengine import Config
cfg = Config.fromfile('./configs/swin_transformer_v2/swinv2-small-w8_16xb64_in1k-256px.py')
# print(cfg.pretty_text)

In [6]:
max_epochs = 100
batch_size = 48

lr_scale_factor = batch_size/(16 * 64)
epoch_scale_factor = max_epochs/cfg.train_cfg.max_epochs

cfg.model.head.num_classes = 10
cfg.data_preprocessor.num_classes = 10
cfg.load_from = './checkpoints/swinv2-small-w8_3rdparty_in1k-256px_20220803-b01a4332.pth'
cfg.work_dir = './work_dir'

cfg.dataset_type = 'CustomDataset'
cfg.train_dataloader.batch_size = batch_size
cfg.train_dataloader.num_workers = 2
cfg.train_dataloader.dataset.type = cfg.dataset_type

cfg.train_dataloader.dataset.data_root = '/kaggle/input/buildings/dataset/train'
del cfg.train_dataloader.dataset['split']



cfg.val_dataloader.batch_size = cfg.train_dataloader.batch_size
cfg.val_dataloader.num_workers = cfg.train_dataloader.num_workers
cfg.val_dataloader.dataset.data_root = '/kaggle/input/buildings/dataset/val'
cfg.val_dataloader.dataset.type = cfg.dataset_type
del cfg.val_dataloader.dataset['split']



cfg.test_dataloader = cfg.val_dataloader
cfg.val_evaluator = dict(type='Accuracy', topk=1)
cfg.test_evaluator = cfg.val_evaluator
cfg.optim_wrapper.optimizer.lr = cfg.optim_wrapper.optimizer.lr * lr_scale_factor



# LinearLR scheduler end epoch
cfg.param_scheduler[0].end = int(cfg.param_scheduler[0].end * epoch_scale_factor)

# CosineAnnealingLR scheduler
cfg.param_scheduler[1].begin = cfg.param_scheduler[0].end

cfg.train_cfg.max_epochs = max_epochs
cfg.train_cfg.val_begin = 20
cfg.train_cfg.val_interval = 5



cfg.default_hooks.checkpoint = dict(type='CheckpointHook', interval=10, max_keep_ckpts=2, save_best='auto')
cfg.default_hooks.logger.interval = 10
cfg.randomness.seed = 2023
#------------------------------------------------------

buildings_config = f'./configs/swin_transformer_v2/swinv2-small-w8_buildings.py'
with open(buildings_config, 'w') as f:
    f.write(cfg.pretty_text)

In [ ]:
!python tools/train.py {buildings_config}

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
11/07 23:40:59 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 2023
    GPU 0: Tesla P100-PCIE-16GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
    PyTorch: 2.0.0
    PyTorch compiling details: PyTorch built with:
  - GCC 11.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6